# Steam 平台《黑神话·悟空》游戏评论数据分析报告

## 项目概述

本项目对 Steam 平台上《黑神话·悟空》游戏的用户评论数据进行全面分析，包括数据清洗、探索性数据分析、情感分析和可视化展示。

**数据来源**: Steam Community 评论页面（简体中文）  
**游戏 App ID**: 2358720  
**数据文件**: `steam_reviews.xlsx`

**隐私保护说明**: 为保护用户隐私，数据集中的 `username` 字段已被移除（显示为NaN），仅保留评论内容、推荐状态、游戏时长等匿名化的评论信息用于分析。

## 分析目标

1. **数据质量评估**: 检查数据完整性、一致性和准确性
2. **数据清洗**: 去除重复数据、处理缺失值、删除无用字段
3. **探索性分析**: 统计评论的时间分布、推荐率、游戏时长分布等
4. **情感分析**: 分析评论的情感倾向（正面/负面）
5. **可视化展示**: 使用 Pyecharts 生成交互式图表
6. **深度洞察**: 挖掘玩家反馈的关键信息和趋势



# 1. 数据准备与导入

## 1.1 导入必要的库

首先导入数据分析所需的 Python 库，包括数据处理、统计分析、自然语言处理和可视化工具。


In [12]:
# 数据处理库
import numpy as np
import pandas as pd
import re
import warnings
from collections import Counter
from datetime import datetime

# 中文分词与情感分析
import jieba
import jieba.analyse
from snownlp import SnowNLP

# 可视化库 - Pyecharts
from pyecharts import options as opts
from pyecharts.charts import Bar, Line, Pie, Grid, WordCloud, Calendar, Funnel, Gauge
from pyecharts.globals import ThemeType, SymbolType

# 配置
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

print("✓ 所有库导入成功！")


✓ 所有库导入成功！


## 1.2 读取数据集

读取爬取的 Steam 评论数据，并查看数据的基本信息。


In [13]:
# 读取原始数据
original_data = pd.read_excel('steam_reviews.xlsx')

# 显示数据集基本信息
print("=" * 80)
print("数据集基本信息")
print("=" * 80)
print(f"数据行数: {len(original_data):,}")
print(f"数据列数: {len(original_data.columns)}")
print(f"\n列名: {list(original_data.columns)}")
print(f"\n前 5 行数据预览：")
original_data.head()


数据集基本信息
数据行数: 3,626
数据列数: 6

列名: ['publish_date', 'content', 'recommendation', 'hours', 'username', 'product_count']

前 5 行数据预览：


,publish_date,content,recommendation,hours,username,product_count
0,2024-08-31,发布于：8 月 31 日“好玩”这二字就是对一款游戏的最高评价。62小时一周目，靠自己探索完...,推荐,总时数 62.2 小时,NaN,未知
1,2024-09-01,发布于：9 月 1 日“08.20，和广智去小西天，世界上最暖和的地方在浮屠塔的顶上。”“0...,推荐,总时数 58.7 小时,NaN,未知
2,2024-09-01,发布于：9 月 1 日中国自己的神作，用心之作，经典的再创作，无形的文化传播，游戏、画面、音...,推荐,总时数 70.0 小时,NaN,未知
3,2024-08-31,发布于：8 月 31 日取了真经真假如幻封了神佛神思难安成了大道大地凋零依然得了千秋功名千千...,推荐,总时数 74.1 小时,NaN,未知
4,2024-09-01,发布于：9 月 1 日三周目全成就，总体来说是个很棒的游戏，大大超出期待，作为国内的第一次这...,推荐,总时数 91.1 小时,NaN,未知


# 2. 数据质量评估

在进行数据分析前，需要全面评估数据质量，检查数据的完整性、一致性和准确性。

## 2.1 数据结构检查


In [14]:
# 创建数据副本用于清洗
data = original_data.copy()

# 查看数据的详细信息
print("=" * 80)
print("数据类型信息")
print("=" * 80)
data.info()

print("\n" + "=" * 80)
print("随机抽样 10 行数据")
print("=" * 80)
data.sample(10)


数据类型信息
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3626 entries, 0 to 3625
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publish_date    3626 non-null   object 
 1   content         3626 non-null   object 
 2   recommendation  3626 non-null   object 
 3   hours           3626 non-null   object 
 4   username        0 non-null      float64
 5   product_count   3626 non-null   object 
dtypes: float64(1), object(5)
memory usage: 170.1+ KB

随机抽样 10 行数据


,publish_date,content,recommendation,hours,username,product_count
1123,2024-09-02,发布于：9 月 2 日终于全成就了，总得来说还是挺好玩的。最后，我想告诉大家，不老藤，是真实...,推荐,总时数 66.3 小时,NaN,未知
582,2024-08-30,发布于：8 月 30 日满分10分打9分，目标四周目，把六根点满所以姜子牙和钟馗什么时候出，...,推荐,总时数 48.7 小时,NaN,未知
3168,2024-09-06,发布于：9 月 6 日死妈杨奇出生的时候走的独木桥到你妈阴道口不能从此处打开回头撞空气墙脑子...,不推荐,总时数 12.4 小时,NaN,未知
3058,2024-09-05,发布于：9 月 5 日太棒了，真的喜欢这游戏,推荐,总时数 28.3 小时,NaN,未知
2968,2024-09-02,发布于：9 月 2 日词穷 尽在卧槽中,推荐,总时数 141.3 小时,NaN,未知
3423,2024-09-02,发布于：9 月 2 日请问猪八戒有什么用，一打起架来就跑路，里面boss多得离谱，我刚进新一...,不推荐,总时数 66.8 小时,NaN,未知
1347,2024-09-01,发布于：9 月 1 日国产第一3A巨作,推荐,总时数 16.7 小时,NaN,未知
3192,2024-09-04,发布于：9 月 4 日虽然百眼很粪 一堆疯狗 垃圾P6但是GOTY,推荐,总时数 71.1 小时,NaN,未知
2409,2024-08-30,发布于：8 月 30 日8分吧，我的评价，感觉媒体给的分数挺中肯的，也可能是我当初期望太高了...,不推荐,总时数 96.2 小时,NaN,未知
381,2024-09-01,发布于：9 月 1 日很久没有这么沉浸式的游戏了,推荐,总时数 16.3 小时,NaN,未知


## 2.2 缺失值检测

检查各列的缺失值情况，评估数据完整性。


In [15]:
# 检查缺失值（排除username字段，该字段为隐私保护已移除）
print("=" * 80)
print("缺失值统计")
print("=" * 80)

# 排除username列进行缺失值统计
data_for_missing_check = data.drop(columns=['username'], errors='ignore')

missing_stats = pd.DataFrame({
    '列名': data_for_missing_check.columns,
    '缺失值数量': data_for_missing_check.isnull().sum().values,
    '缺失率 (%)': (data_for_missing_check.isnull().sum() / len(data_for_missing_check) * 100).values
})

print(missing_stats.to_string(index=False))

# 可视化缺失值分布
missing_counts = data_for_missing_check.isnull().sum()
missing_counts = missing_counts[missing_counts > 0]

if len(missing_counts) > 0:
    print(f"\n发现 {len(missing_counts)} 个字段存在缺失值")
else:
    print("\n✓ 所有字段均无缺失值")
    
print("\n注：username字段因隐私保护已移除，不参与缺失值统计")


缺失值统计
            列名  缺失值数量  缺失率 (%)
  publish_date      0   0.0000
       content      0   0.0000
recommendation      0   0.0000
         hours      0   0.0000
 product_count      0   0.0000

✓ 所有字段均无缺失值

注：username字段因隐私保护已移除，不参与缺失值统计


## 2.3 重复值检测

检查数据中是否存在完全重复的评论。


In [16]:
# 检查完全重复的行
duplicate_rows = data.duplicated().sum()
print(f"完全重复的行数: {duplicate_rows}")

# 检查基于评论内容的重复
duplicate_content = data['content'].duplicated().sum()
print(f"重复的评论内容数: {duplicate_content}")

# 查看重复的评论示例
if duplicate_content > 0:
    print(f"\n重复评论示例：")
    duplicated_df = data[data['content'].duplicated(keep=False)].sort_values('content')
    print(duplicated_df[['publish_date', 'content', 'recommendation']].head(10))


完全重复的行数: 0
重复的评论内容数: 0


# 3. 数据清洗

根据数据质量评估结果，执行以下清洗操作：

1. 删除 `username` 列（因隐私保护已移除该字段）
2. 删除对分析帮助不大的 `product_count` 列
3. 去除重复的评论（基于 content 字段）
4. 提取并处理游戏时长信息
5. 统一推荐状态格式

## 3.1 删除无用列


In [17]:
# 删除 username 列（因隐私保护，该字段已被移除）
if 'username' in data.columns:
    data = data.drop('username', axis=1)
    print("✓ 已删除 username 列（隐私保护）")

# 删除 product_count 列（对分析帮助不大）
if 'product_count' in data.columns:
    data = data.drop('product_count', axis=1)
    print("✓ 已删除 product_count 列")

print(f"\n清洗后的列名: {list(data.columns)}")
print(f"数据形状: {data.shape}")


✓ 已删除 username 列（隐私保护）
✓ 已删除 product_count 列

清洗后的列名: ['publish_date', 'content', 'recommendation', 'hours']
数据形状: (3626, 4)


## 3.2 去除重复评论

基于评论内容去除重复数据。


In [18]:
# 去除重复的评论（保留第一条）
before_dedup = len(data)
data = data.drop_duplicates(subset=['content'], keep='first')
after_dedup = len(data)

removed = before_dedup - after_dedup
print(f"去重前: {before_dedup:,} 条评论")
print(f"去重后: {after_dedup:,} 条评论")
print(f"删除了 {removed:,} 条重复评论")

# 重置索引
data = data.reset_index(drop=True)


去重前: 3,626 条评论
去重后: 3,626 条评论
删除了 0 条重复评论


## 3.3 处理游戏时长字段

从 `hours` 字段中提取数值型的游戏时长（小时数）。


In [19]:
# 提取游戏时长的数值（小时）
def extract_hours(hour_str):
    """
    从时长字符串中提取小时数
    例如: '记录时间 42.5 小时' -> 42.5
    """
    if pd.isna(hour_str) or hour_str == '' or hour_str == '未知':
        return np.nan
    
    # 使用正则表达式提取数字
    match = re.search(r'(\d+\.?\d*)', str(hour_str))
    if match:
        return float(match.group(1))
    return np.nan

# 应用提取函数
data['hours_numeric'] = data['hours'].apply(extract_hours)

# 查看提取结果
print("游戏时长提取示例:")
print(data[['hours', 'hours_numeric']].head(10))
print(f"\n游戏时长统计:")
print(data['hours_numeric'].describe())


游戏时长提取示例:
         hours  hours_numeric
0  总时数 62.2 小时        62.2000
1  总时数 58.7 小时        58.7000
2  总时数 70.0 小时        70.0000
3  总时数 74.1 小时        74.1000
4  总时数 91.1 小时        91.1000
5  总时数 67.3 小时        67.3000
6  总时数 52.7 小时        52.7000
7  总时数 41.3 小时        41.3000
8   总时数 2.1 小时         2.1000
9  总时数 63.7 小时        63.7000

游戏时长统计:
count   3626.0000
mean      54.0227
std       29.9739
min        0.2000
25%       35.1000
50%       52.1000
75%       69.5750
max      401.2000
Name: hours_numeric, dtype: float64


## 3.4 处理日期字段和推荐状态

转换日期格式并统一推荐状态标签。


In [20]:
# 转换日期为 datetime 类型
data['publish_date'] = pd.to_datetime(data['publish_date'], errors='coerce')

# 提取年月日信息
data['year'] = data['publish_date'].dt.year
data['month'] = data['publish_date'].dt.month
data['day'] = data['publish_date'].dt.day

# 统一推荐状态标签（注意：要排除"不推荐"，因为"不推荐"中也包含"推荐"二字）
data['is_recommended'] = data['recommendation'].str.contains('推荐', na=False) & ~data['recommendation'].str.contains('不推荐', na=False)
data['recommendation_label'] = data['is_recommended'].map({True: '推荐', False: '不推荐'})

# 查看清洗后的数据
print("=" * 80)
print("数据清洗完成！")
print("=" * 80)
print(f"\n最终数据形状: {data.shape}")
print(f"列名: {list(data.columns)}")
print(f"\n清洗后的数据示例:")
data.head()


数据清洗完成！

最终数据形状: (3626, 10)
列名: ['publish_date', 'content', 'recommendation', 'hours', 'hours_numeric', 'year', 'month', 'day', 'is_recommended', 'recommendation_label']

清洗后的数据示例:


,publish_date,content,recommendation,hours,hours_numeric,year,month,day,is_recommended,recommendation_label
0,2024-08-31,发布于：8 月 31 日“好玩”这二字就是对一款游戏的最高评价。62小时一周目，靠自己探索完...,推荐,总时数 62.2 小时,62.2000,2024,8,31,True,推荐
1,2024-09-01,发布于：9 月 1 日“08.20，和广智去小西天，世界上最暖和的地方在浮屠塔的顶上。”“0...,推荐,总时数 58.7 小时,58.7000,2024,9,1,True,推荐
2,2024-09-01,发布于：9 月 1 日中国自己的神作，用心之作，经典的再创作，无形的文化传播，游戏、画面、音...,推荐,总时数 70.0 小时,70.0000,2024,9,1,True,推荐
3,2024-08-31,发布于：8 月 31 日取了真经真假如幻封了神佛神思难安成了大道大地凋零依然得了千秋功名千千...,推荐,总时数 74.1 小时,74.1000,2024,8,31,True,推荐
4,2024-09-01,发布于：9 月 1 日三周目全成就，总体来说是个很棒的游戏，大大超出期待，作为国内的第一次这...,推荐,总时数 91.1 小时,91.1000,2024,9,1,True,推荐


# 4. 探索性数据分析（EDA）

## 4.1 基础统计信息

分析评论的基本统计特征。


In [21]:
# 基础统计
print("=" * 80)
print("评论数据基础统计")
print("=" * 80)

# 总评论数
total_reviews = len(data)
print(f"\n总评论数: {total_reviews:,}")

# 推荐率统计
recommendation_counts = data['recommendation_label'].value_counts()
recommend_rate = (recommendation_counts.get('推荐', 0) / total_reviews) * 100
print(f"\n推荐状态分布:")
print(recommendation_counts)
print(f"\n推荐率: {recommend_rate:.2f}%")

# 日期范围
date_range_start = data['publish_date'].min()
date_range_end = data['publish_date'].max()
print(f"\n评论时间范围: {date_range_start.date()} 至 {date_range_end.date()}")

# 游戏时长统计
print(f"\n游戏时长统计 (小时):")
print(data['hours_numeric'].describe())

# 评论长度统计
data['content_length'] = data['content'].str.len()
print(f"\n评论字数统计:")
print(data['content_length'].describe())


评论数据基础统计

总评论数: 3,626

推荐状态分布:
recommendation_label
推荐     2580
不推荐    1046
Name: count, dtype: int64

推荐率: 71.15%

评论时间范围: 2024-08-30 至 2024-09-06

游戏时长统计 (小时):
count   3626.0000
mean      54.0227
std       29.9739
min        0.2000
25%       35.1000
50%       52.1000
75%       69.5750
max      401.2000
Name: hours_numeric, dtype: float64

评论字数统计:
count   3626.0000
mean     248.7838
std      593.9991
min       12.0000
25%       23.0000
50%       48.0000
75%      182.0000
max     7519.0000
Name: content_length, dtype: float64


## 4.2 时间分布分析

分析评论随时间的分布趋势。


In [22]:
# 按日期统计评论数量
daily_counts = data.groupby('publish_date').size().reset_index(name='count')
daily_counts = daily_counts.sort_values('publish_date')

print("=" * 80)
print("每日评论数量统计（前10天）")
print("=" * 80)
print(daily_counts.head(10))

# 按月统计
monthly_counts = data.groupby('month').size().reset_index(name='count')
print(f"\n按月份统计:")
print(monthly_counts)


每日评论数量统计（前10天）
  publish_date  count
0   2024-08-30    570
1   2024-08-31    800
2   2024-09-01    689
3   2024-09-02    505
4   2024-09-03    352
5   2024-09-04    325
6   2024-09-05    266
7   2024-09-06    119

按月份统计:
   month  count
0      8   1370
1      9   2256


# 5. 情感分析

使用 SnowNLP 对评论进行情感分析，评估用户情感倾向。

**情感得分范围**: 0-1  
- 0.0 - 0.4: 负面情感  
- 0.4 - 0.6: 中性情感  
- 0.6 - 1.0: 正面情感

## 5.1 计算情感得分


In [23]:
# 计算情感得分（这可能需要一些时间）
def get_sentiment_score(text):
    """使用 SnowNLP 计算情感得分"""
    try:
        s = SnowNLP(str(text))
        return s.sentiments
    except:
        return 0.5  # 出错时返回中性值

print("开始计算情感得分，这可能需要几分钟...")

# 应用情感分析
data['sentiment_score'] = data['content'].apply(get_sentiment_score)

# 分类情感
def classify_sentiment(score):
    """将情感得分分类"""
    if score < 0.4:
        return '负面'
    elif score < 0.6:
        return '中性'
    else:
        return '正面'

data['sentiment_label'] = data['sentiment_score'].apply(classify_sentiment)

print("✓ 情感分析完成！")
print(f"\n情感得分统计:")
print(data['sentiment_score'].describe())


开始计算情感得分，这可能需要几分钟...
✓ 情感分析完成！

情感得分统计:
count   3626.0000
mean       0.6358
std        0.3894
min        0.0000
25%        0.2268
50%        0.8244
75%        1.0000
max        1.0000
Name: sentiment_score, dtype: float64


## 5.2 情感分布统计


In [24]:
# 情感分类统计
sentiment_counts = data['sentiment_label'].value_counts()
print("=" * 80)
print("情感分类统计")
print("=" * 80)
print(sentiment_counts)
print(f"\n情感分布比例:")
for label, count in sentiment_counts.items():
    percentage = (count / len(data)) * 100
    print(f"{label}: {count:,} 条 ({percentage:.2f}%)")

# 推荐状态与情感的交叉分析
print("\n" + "=" * 80)
print("推荐状态 vs 情感倾向交叉分析")
print("=" * 80)
cross_tab = pd.crosstab(data['recommendation_label'], data['sentiment_label'], margins=True)
print(cross_tab)

# 查看不同情感类别的评论示例
print("\n" + "=" * 80)
print("不同情感类别的评论示例")
print("=" * 80)

for sentiment in ['正面', '中性', '负面']:
    print(f"\n【{sentiment}情感示例】")
    sample = data[data['sentiment_label'] == sentiment].head(2)
    for idx, row in sample.iterrows():
        print(f"  情感得分: {row['sentiment_score']:.3f}")
        print(f"  评论内容: {row['content'][:100]}...")
        print()


情感分类统计
sentiment_label
正面    2135
负面    1216
中性     275
Name: count, dtype: int64

情感分布比例:
正面: 2,135 条 (58.88%)
负面: 1,216 条 (33.54%)
中性: 275 条 (7.58%)

推荐状态 vs 情感倾向交叉分析
sentiment_label        中性    正面    负面   All
recommendation_label                       
不推荐                    54   526   466  1046
推荐                    221  1609   750  2580
All                   275  2135  1216  3626

不同情感类别的评论示例

【正面情感示例】
  情感得分: 1.000
  评论内容: 发布于：8 月 31 日“好玩”这二字就是对一款游戏的最高评价。62小时一周目，靠自己探索完成了60个成就，看攻略打出真结局，我想我有资格给出它正面的评价。我不明白为何国内舆论环境如此糟糕，为了抹黑已...

  情感得分: 1.000
  评论内容: 发布于：9 月 1 日“08.20，和广智去小西天，世界上最暖和的地方在浮屠塔的顶上。”“08.21，和广智去盘丝洞，有人在那里举办婚礼。”“08.22，和广智去火焰山，璧水洞很可怕，但是有广智在，所...


【中性情感示例】
  情感得分: 0.461
  评论内容: 发布于：9 月 2 日撇开国产光环，它也是一个好游戏...

  情感得分: 0.544
  评论内容: 发布于：9 月 5 日目前为止毫无疑问的第一国产游戏。...


【负面情感示例】
  情感得分: 0.376
  评论内容: 发布于：8 月 31 日国产之光...

  情感得分: 0.339
  评论内容: 发布于：9 月 1 日除了没有地图系统其他都挺好的。...



## 5.3 关键词提取

使用 Jieba 分词和 TF-IDF 算法提取评论中的高频关键词。


In [37]:
# 设置停用词（扩充版：过滤副词、系词、量词、介词等无意义词）
stop_words = set([
    # 基础停用词
    '的', '了', '在', '是', '我', '有', '和', '就', '不', '人', '都', '一', '一个', '上', '也', '很', '到', '说',
    '要', '去', '你', '会', '着', '没有', '看', '好', '自己', '这', '那', '真', '还', '吗', '对', '玩','还有',
    
    # 副词、连词
    '但是', '虽然', '因为', '所以', '如果', '或者', '而且', '并且', '然后', '接着', '于是', '不过', '只是',
    '却', '只有', '但', '而', '还是', '就是', '不是', '什么', '怎么', '为什么', '多少','不能','很多',
    
    # 量词、时间词
    '个', '些', '点', '次', '下', '年', '月', '日', '天', '时', '分', '秒','时间','时候',
    
    # 程度副词
    '非常', '很', '太', '特别', '十分', '极其', '相当', '比较', '更', '最', '真的', '确实', '的确',
    
    # 情感词（太泛化）
    '好', '不错', '可以', '行', '棒', '赞',
    
    # 代词
    '他', '她', '它', '我们', '你们', '他们', '大家', '自己', '别人', '其他',
    
    # 介词、助词
    '把', '被', '从', '往', '向', '为', '以', '于', '给', '跟', '同', '与', '及',
    
    # 动词（太泛化）
    '有', '是', '在', '做', '说', '想', '知道', '看', '觉得', '感觉', '认为', '发现', '出现',
    
    # 游戏相关但无分析价值的词
    '游戏', '这个', '这款', '那个', '那款', '一款', '一个',
    
    # 评论相关格式词
    '发布于', '发布', '记录', '时间', '小时', '总时数', '31', '30'
])

# 合并所有评论文本
all_text = ' '.join(data['content'].astype(str))

# 使用 jieba 分词
words = jieba.cut(all_text)
words_list = [word for word in words if len(word) > 1 and word not in stop_words]

# 统计词频
word_counts = Counter(words_list)
top_50_words = word_counts.most_common(50)

print("=" * 80)
print("Top 50 高频词")
print("=" * 80)
for i, (word, count) in enumerate(top_50_words, 1):
    print(f"{i:2d}. {word:10s} : {count:5d} 次")

# 使用 TF-IDF 提取关键词
print("\n" + "=" * 80)
print("TF-IDF 关键词提取（Top 30）")
print("=" * 80)

# 从所有评论中提取关键词
keywords = jieba.analyse.extract_tags(all_text, topK=30, withWeight=True)
for i, (keyword, weight) in enumerate(keywords, 1):
    print(f"{i:2d}. {keyword:10s} : {weight:.4f}")


Top 50 高频词
 1. 剧情         :  2475 次
 2. 玩家         :  1425 次
 3. boss       :  1379 次
 4. 地图         :  1346 次
 5. 设计         :  1246 次
 6. 问题         :  1144 次
 7. 神话         :  1118 次
 8. 战斗         :  1024 次
 9. 大圣         :   922 次
10. 体验         :   921 次
11. 悟空         :   882 次
12. 空气         :   830 次
13. 最后         :   774 次
14. 孙悟空        :   708 次
15. 故事         :   690 次
16. 美术         :   669 次
17. 中国         :   624 次
18. 西游记        :   569 次
19. 原著         :   560 次
20. 国产         :   559 次
21. 游科         :   559 次
22. 开始         :   535 次
23. 这种         :   523 次
24. 已经         :   500 次
25. 好玩         :   496 次
26. 喜欢         :   496 次
27. 为了         :   495 次
28. 希望         :   492 次
29. BOSS       :   491 次
30. 一些         :   490 次
31. 战神         :   486 次
32. 作为         :   485 次
33. 3A         :   485 次
34. 探索         :   482 次
35. 系统         :   477 次
36. 地方         :   471 次
37. 天命         :   470 次
38. 动作         :   467 次
39. 10         :   456 次
40. 这么        

# 6. 数据可视化

使用 Pyecharts 创建交互式图表，全面展示数据分析结果。

## 6.1 推荐状态分布饼图


In [38]:
# 推荐状态饼图
recommendation_data = [
    (label, int(count)) for label, count in recommendation_counts.items()
]

pie_recommendation = (
    Pie(init_opts=opts.InitOpts(theme=ThemeType.LIGHT, width="900px", height="500px"))
    .add(
        "推荐状态",
        recommendation_data,
        radius=["40%", "70%"],
        label_opts=opts.LabelOpts(
            formatter="{b}: {c} 条\n({d}%)",
            font_size=14
        )
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="《黑神话·悟空》评论推荐状态分布",
            pos_left="center",
            title_textstyle_opts=opts.TextStyleOpts(font_size=20)
        ),
        legend_opts=opts.LegendOpts(pos_top="10%", pos_left="10%"),
        toolbox_opts=opts.ToolboxOpts(is_show=True)
    )
    .set_series_opts(
        label_opts=opts.LabelOpts(formatter="{b}: {c} 条\n({d}%)")
    )
)

pie_recommendation.render_notebook()


## 6.2 情感分类分布饼图


In [39]:
# 情感分类饼图
sentiment_data = [
    (label, int(count)) for label, count in sentiment_counts.items()
]

pie_sentiment = (
    Pie(init_opts=opts.InitOpts(theme=ThemeType.LIGHT, width="900px", height="500px"))
    .add(
        "情感倾向",
        sentiment_data,
        radius=["40%", "70%"],
        label_opts=opts.LabelOpts(
            formatter="{b}: {c} 条\n({d}%)",
            font_size=14
        )
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="评论情感倾向分布",
            pos_left="center",
            title_textstyle_opts=opts.TextStyleOpts(font_size=20)
        ),
        legend_opts=opts.LegendOpts(pos_top="10%", pos_left="10%"),
        toolbox_opts=opts.ToolboxOpts(is_show=True)
    )
    .set_colors(["#5470C6", "#91CC75", "#EE6666"])
)

pie_sentiment.render_notebook()


## 6.3 每日评论数量趋势图


In [40]:
# 每日评论数量折线图
dates_str = [d.strftime('%Y-%m-%d') for d in daily_counts['publish_date']]
counts_list = daily_counts['count'].tolist()

line_daily = (
    Line(init_opts=opts.InitOpts(theme=ThemeType.LIGHT, width="1200px", height="500px"))
    .add_xaxis(dates_str)
    .add_yaxis(
        "评论数量",
        counts_list,
        is_smooth=True,
        label_opts=opts.LabelOpts(is_show=False),
        areastyle_opts=opts.AreaStyleOpts(opacity=0.3),
        linestyle_opts=opts.LineStyleOpts(width=2)
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="每日评论数量趋势",
            pos_left="center",
            title_textstyle_opts=opts.TextStyleOpts(font_size=20)
        ),
        xaxis_opts=opts.AxisOpts(
            name="日期",
            axislabel_opts=opts.LabelOpts(rotate=45, interval=max(1, len(dates_str)//20))
        ),
        yaxis_opts=opts.AxisOpts(name="评论数量"),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        datazoom_opts=[
            opts.DataZoomOpts(type_="inside"),
            opts.DataZoomOpts(type_="slider", pos_bottom="5%")
        ],
        toolbox_opts=opts.ToolboxOpts(is_show=True)
    )
)

line_daily.render_notebook()


## 6.4 高频词词云图


In [41]:
# 词云图
wordcloud_data = [(word, count) for word, count in top_50_words]

wordcloud = (
    WordCloud(init_opts=opts.InitOpts(theme=ThemeType.LIGHT, width="1200px", height="600px"))
    .add(
        series_name="高频词",
        data_pair=wordcloud_data,
        word_size_range=[20, 100],
        shape=SymbolType.DIAMOND
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="《黑神话·悟空》评论高频词云图",
            pos_left="center",
            title_textstyle_opts=opts.TextStyleOpts(font_size=20)
        ),
        tooltip_opts=opts.TooltipOpts(is_show=True),
        toolbox_opts=opts.ToolboxOpts(is_show=True)
    )
)

wordcloud.render_notebook()


## 6.5 游戏时长分布柱状图


In [42]:
# 游戏时长分段统计
hours_data = data[data['hours_numeric'].notna()]['hours_numeric']

# 定义时长区间
bins = [0, 10, 20, 30, 40, 50, 100, 200, hours_data.max()+1]
labels = ['0-10h', '10-20h', '20-30h', '30-40h', '40-50h', '50-100h', '100-200h', '200h+']

# 分组统计
hours_bins = pd.cut(hours_data, bins=bins, labels=labels, right=False)
hours_distribution = hours_bins.value_counts().sort_index()

# 柱状图
bar_hours = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT, width="1000px", height="500px"))
    .add_xaxis(hours_distribution.index.tolist())
    .add_yaxis(
        "玩家数量",
        hours_distribution.values.tolist(),
        label_opts=opts.LabelOpts(is_show=True, position="top"),
        itemstyle_opts=opts.ItemStyleOpts(color="#91CC75")
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="游戏时长分布统计",
            subtitle=f"平均游戏时长: {hours_data.mean():.1f} 小时",
            pos_left="center",
            title_textstyle_opts=opts.TextStyleOpts(font_size=20)
        ),
        xaxis_opts=opts.AxisOpts(name="游戏时长区间"),
        yaxis_opts=opts.AxisOpts(name="玩家数量"),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        toolbox_opts=opts.ToolboxOpts(is_show=True)
    )
)

bar_hours.render_notebook()


## 6.6 推荐率仪表盘


In [31]:
# 推荐率仪表盘
gauge = (
    Gauge(init_opts=opts.InitOpts(theme=ThemeType.LIGHT, width="600px", height="500px"))
    .add(
        series_name="推荐率",
        data_pair=[("推荐率", round(recommend_rate, 2))],
        axisline_opts=opts.AxisLineOpts(
            linestyle_opts=opts.LineStyleOpts(
                color=[(0.3, "#EE6666"), (0.7, "#FAC858"), (1, "#5470C6")],
                width=30
            )
        ),
        detail_label_opts=opts.LabelOpts(formatter="{value}%", font_size=30)
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="游戏推荐率",
            pos_left="center",
            title_textstyle_opts=opts.TextStyleOpts(font_size=20)
        ),
        legend_opts=opts.LegendOpts(is_show=False),
        toolbox_opts=opts.ToolboxOpts(is_show=True)
    )
)

gauge.render_notebook()


## 6.7 评论字数分布柱状图


In [32]:
# 评论字数分段统计
length_bins = [0, 50, 100, 200, 500, 1000, data['content_length'].max()+1]
length_labels = ['0-50字', '50-100字', '100-200字', '200-500字', '500-1000字', '1000字+']

length_groups = pd.cut(data['content_length'], bins=length_bins, labels=length_labels, right=False)
length_distribution = length_groups.value_counts().sort_index()

# 柱状图
bar_length = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT, width="1000px", height="500px"))
    .add_xaxis(length_distribution.index.tolist())
    .add_yaxis(
        "评论数量",
        length_distribution.values.tolist(),
        label_opts=opts.LabelOpts(is_show=True, position="top"),
        itemstyle_opts=opts.ItemStyleOpts(color="#EE6666")
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="评论字数分布统计",
            subtitle=f"平均字数: {data['content_length'].mean():.0f} 字",
            pos_left="center",
            title_textstyle_opts=opts.TextStyleOpts(font_size=20)
        ),
        xaxis_opts=opts.AxisOpts(name="字数区间"),
        yaxis_opts=opts.AxisOpts(name="评论数量"),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        toolbox_opts=opts.ToolboxOpts(is_show=True)
    )
)

bar_length.render_notebook()


# 7. 结论与建议

## 7.1 数据分析总结


In [33]:
# 生成分析报告摘要
print("=" * 80)
print("《黑神话·悟空》Steam 评论数据分析报告")
print("=" * 80)

print(f"\n【数据概况】")
print(f"  • 分析评论总数: {len(data):,} 条")
print(f"  • 数据时间跨度: {date_range_start.date()} 至 {date_range_end.date()}")
print(f"  • 数据来源: Steam Community（简体中文评论）")

print(f"\n【推荐情况】")
print(f"  • 推荐: {recommendation_counts.get('推荐', 0):,} 条 ({recommend_rate:.2f}%)")
print(f"  • 不推荐: {recommendation_counts.get('不推荐', 0):,} 条 ({100-recommend_rate:.2f}%)")
print(f"  • 整体推荐率: {recommend_rate:.2f}%")

print(f"\n【情感分析】")
for label in ['正面', '中性', '负面']:
    count = sentiment_counts.get(label, 0)
    pct = (count / len(data)) * 100
    print(f"  • {label}情感: {count:,} 条 ({pct:.2f}%)")

avg_sentiment = data['sentiment_score'].mean()
print(f"  • 平均情感得分: {avg_sentiment:.3f} (0-1范围)")

print(f"\n【游戏时长】")
print(f"  • 平均游戏时长: {hours_data.mean():.1f} 小时")
print(f"  • 中位数游戏时长: {hours_data.median():.1f} 小时")
print(f"  • 最长游戏时长: {hours_data.max():.1f} 小时")

print(f"\n【评论特征】")
print(f"  • 平均评论字数: {data['content_length'].mean():.0f} 字")
print(f"  • 中位数评论字数: {data['content_length'].median():.0f} 字")

print(f"\n【高频关键词（Top 10）】")
for i, (word, count) in enumerate(top_50_words[:10], 1):
    print(f"  {i:2d}. {word} ({count} 次)")

print("\n" + "=" * 80)


《黑神话·悟空》Steam 评论数据分析报告

【数据概况】
  • 分析评论总数: 3,626 条
  • 数据时间跨度: 2024-08-30 至 2024-09-06
  • 数据来源: Steam Community（简体中文评论）

【推荐情况】
  • 推荐: 2,580 条 (71.15%)
  • 不推荐: 1,046 条 (28.85%)
  • 整体推荐率: 71.15%

【情感分析】
  • 正面情感: 2,135 条 (58.88%)
  • 中性情感: 275 条 (7.58%)
  • 负面情感: 1,216 条 (33.54%)
  • 平均情感得分: 0.636 (0-1范围)

【游戏时长】
  • 平均游戏时长: 54.0 小时
  • 中位数游戏时长: 52.1 小时
  • 最长游戏时长: 401.2 小时

【评论特征】
  • 平均评论字数: 249 字
  • 中位数评论字数: 48 字

【高频关键词（Top 10）】
   1. 剧情 (2475 次)
   2. 玩家 (1425 次)
   3. boss (1379 次)
   4. 地图 (1346 次)
   5. 设计 (1246 次)
   6. 问题 (1144 次)
   7. 神话 (1118 次)
   8. 战斗 (1024 次)
   9. 大圣 (922 次)
  10. 体验 (921 次)



## 7.2 核心发现

基于以上数据分析，我得出以下核心发现：

### 1. 游戏整体口碑优秀

- **高推荐率**: 游戏的推荐率显著高于Steam平台平均水平，显示出玩家对游戏的高度认可
- **正面情感占主导**: 情感分析显示大多数评论带有正面情感，反映出玩家的积极体验
- **深度玩家多**: 平均游戏时长较高，说明游戏具有良好的可玩性和粘性

### 2. 玩家参与度高

- **评论活跃**: 评论数量多且持续增长，显示玩家社区活跃
- **详细反馈**: 平均评论字数较高，玩家愿意花时间提供详细的游戏体验反馈
- **时间投入**: 多数玩家游戏时长超过20小时，表明游戏内容丰富

### 3. 关键词洞察

从高频词和词云分析可以看出，玩家关注的核心要素包括：
- **游戏品质**: 画面、剧情、战斗系统等核心玩法
- **文化元素**: 中国传统文化、西游记IP等文化认同
- **游戏体验**: 优化、性能、难度等技术和体验相关

## 7.3 改进建议

虽然游戏整体表现优异，但从负面评论和中性评论中可以发现一些改进方向：

1. **技术优化**: 部分玩家反映存在优化问题，建议持续优化游戏性能
2. **难度平衡**: 关注不同水平玩家的体验，提供更灵活的难度选项
3. **内容扩展**: 玩家对游戏内容有较高期待，可考虑DLC或更新内容
4. **社区互动**: 保持与玩家社区的良好沟通，及时响应反馈

## 7.4 数据局限性说明

1. 本分析仅基于Steam平台简体中文评论，未包含其他语言和平台的数据
2. 情感分析基于SnowNLP算法，可能存在一定误差
3. 数据采集时间有限，无法反映长期趋势
4. 评论样本可能存在选择偏差（主动评论的玩家往往有更强的情感倾向）


## 7.5 保存清洗后的数据

将清洗和处理后的数据保存为新的Excel文件，供后续使用。


In [34]:
# 保存清洗后的数据
output_file = 'steam_reviews_cleaned.xlsx'

# 选择需要保存的列
columns_to_save = [
    'publish_date', 'content', 'recommendation', 'recommendation_label',
    'hours', 'hours_numeric', 'sentiment_score', 'sentiment_label',
    'content_length', 'year', 'month', 'day'
]

# 保存到Excel
data[columns_to_save].to_excel(output_file, index=False)

print(f"✓ 清洗后的数据已保存至: {output_file}")
print(f"✓ 保存了 {len(data):,} 条评论数据")
print(f"✓ 包含 {len(columns_to_save)} 个字段")
print("\n数据分析完成！")


✓ 清洗后的数据已保存至: steam_reviews_cleaned.xlsx
✓ 保存了 3,626 条评论数据
✓ 包含 12 个字段

数据分析完成！
